# Supervised Learning Music Genre Classification

## Import Statements

Start by importing necessary libraries.

In [169]:
import os
import librosa
import pandas as pd
from pydub import AudioSegment
import tempfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split

## Helper Functions

Convert to MP3 function.

In [170]:
def convert_mp3_to_wav(mp3_file):
    try:
        sound = AudioSegment.from_mp3(mp3_file)
        wav_file = tempfile.mktemp(suffix='.wav')
        sound.export(wav_file, format="wav")
        return wav_file
    except Exception as e:
        print(f"Error converting {mp3_file} to WAV: {e}")
        return None

Feature extraction from a segment. 

In [152]:
# Function to extract features from an audio segment
def extract_features_from_segment(y, sr, start_time, end_time):
    segment = y[start_time:end_time]

    # Compute chroma feature from the waveform and sample rate
    chroma_stft = librosa.feature.chroma_stft(y=segment, sr=sr)
    # Compute Root Mean Square (RMS) energy for each frame
    rms = librosa.feature.rms(y=segment)
    # Compute spectral centroid, which indicates where the center of mass for a sound is located
    spectral_centroid = librosa.feature.spectral_centroid(y=segment, sr=sr)
    # Compute spectral bandwidth, which is a measure of the width of the band of frequencies
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=segment, sr=sr)
    # Compute spectral rolloff point, which is the frequency below which a specified percentage of the total spectral energy lies
    rolloff = librosa.feature.spectral_rolloff(y=segment, sr=sr)
    # Compute zero crossing rate, which is the rate at which the signal changes sign
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=segment)
    # Compute harmony and perceptr (percussive) components of the audio
    harmony, perceptr = librosa.effects.hpss(segment)
    # Compute tempo (beats per minute)
    tempo, _ = librosa.beat.beat_track(y=segment, sr=sr)
    # Compute Mel-frequency cepstral coefficients (MFCCs)
    mfcc = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=20)

    # Aggregate the features into a dictionary
    features = {
        'chroma_stft_mean': chroma_stft.mean() if chroma_stft.size else 0,
        'chroma_stft_var': chroma_stft.var() if chroma_stft.size else 0,
        'rms_mean': rms.mean() if rms.size else 0,
        'rms_var': rms.var() if rms.size else 0,
        'spectral_centroid_mean': spectral_centroid.mean() if spectral_centroid.size else 0,
        'spectral_centroid_var': spectral_centroid.var() if spectral_centroid.size else 0,
        'spectral_bandwidth_mean': spectral_bandwidth.mean() if spectral_bandwidth.size else 0,
        'spectral_bandwidth_var': spectral_bandwidth.var() if spectral_bandwidth.size else 0,
        'rolloff_mean': rolloff.mean() if rolloff.size else 0,
        'rolloff_var': rolloff.var() if rolloff.size else 0,
        'zero_crossing_rate_mean': zero_crossing_rate.mean() if zero_crossing_rate.size else 0,
        'zero_crossing_rate_var': zero_crossing_rate.var() if zero_crossing_rate.size else 0,
        'harmony_mean': harmony.mean() if harmony.size else 0,
        'harmony_var': harmony.var() if harmony.size else 0,
        'perceptr_mean': perceptr.mean() if perceptr.size else 0,
        'perceptr_var': perceptr.var() if perceptr.size else 0,
        'tempo': tempo,
    }

    for i in range(1, 21):
        features[f'mfcc{i}_mean'] = mfcc[i-1].mean() if mfcc.shape[0] >= i else 0
        features[f'mfcc{i}_var'] = mfcc[i-1].var() if mfcc.shape[0] >= i else 0

    return features


Segment data and call feature extraction.

In [171]:
# Function to extract features from an audio file
def extract_features(audio_file, segment_duration=3):
    try:
        # Load the audio file
        y, sr = librosa.load(audio_file, sr=None)
        total_duration = len(y) / sr
        segment_length = int(sr * segment_duration)

        features_list = []

        for start in range(0, len(y), segment_length):
            end = start + segment_length
            if end <= len(y):
                segment_features = extract_features_from_segment(y, sr, start, end)
                segment_features['filename'] = os.path.basename(audio_file)
                segment_features['start'] = start / sr
                segment_features['end'] = end / sr
                features_list.append(segment_features)

        return features_list

    except Exception as e:
        print(f"Error extracting features from {audio_file}: {e}")
        return []

## Define Data Paths

Specify the paths to the CSV files containing data.

In [154]:
# List of genres
genres = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]

# Base folder containing genre subfolders
base_folder_path = '/Users/lchilly/Desktop/Astra/genres_original'

## Process Input Audio Files

Process input files and label the data. 

In [155]:
# Main function to process all audio files in a folder
def process_audio_folder(folder_path, genre_label):
    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav') or filename.endswith('.mp3'):
            file_path = os.path.join(folder_path, filename)
            if filename.endswith('.mp3'):
                try:
                    file_path = convert_mp3_to_wav(file_path)
                except Exception as e:
                    print(f"Error converting {file_path} to WAV: {e}")
                    continue
            features_list = extract_features(file_path)
            for features in features_list:
                features['genre'] = genre_label
                results.append(features)
    return results

## Create Singular CSV

Aggregate results of the feature extraction into a singular CSV.

In [156]:
# Collect results from all genres
all_results = []

# Process each genre
for genre in genres:
    print(f"Processing genre: {genre}")
    folder_path = os.path.join(base_folder_path, genre)
    genre_results = process_audio_folder(folder_path, genre)
    all_results.extend(genre_results)
    print(f"Completed processing genre: {genre}")

# Write all results to a single CSV file
print("Writing results to CSV file...")
df = pd.DataFrame(all_results)
csv_file_path = '/Users/lchilly/Desktop/Astra/music-genre-classification/all_genres_audio_features.csv'
df.to_csv(csv_file_path, index=False)
print("CSV file generation completed.")

# I (simon) mainly added this for testing just so I could see when everything is done
print("Feature extraction and CSV generation completed for all genres.")

df.head()

Processing genre: blues


KeyboardInterrupt: 

## Examine Data

Gain insights into data. 

In [172]:
# Get summary statistics
df.describe()


,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,start,end
count,9981.000000,9981.000000,9981.000000,9.981000e+03,9981.000000,9.981000e+03,9981.000000,9.981000e+03,9981.000000,9.981000e+03,...,9981.000000,9981.000000,9981.000000,9981.000000,9981.000000,9981.000000,9981.000000,9981.000000,9981.000000,9981.000000
mean,0.379964,0.084882,0.130039,2.672433e-03,2201.910957,4.159255e+05,2244.562460,1.183121e+05,4571.568401,1.623468e+06,...,-4.193187,51.838395,0.724376,52.343694,-2.497094,54.811697,-0.929246,57.142098,13.487827,16.487827
std,0.090624,0.009675,0.068168,3.561532e-03,750.540439,4.339675e+05,541.420376,1.002501e+05,1639.481644,1.482634e+06,...,5.668784,36.301829,5.175797,38.067790,5.107192,41.505894,5.247202,46.342797,8.611614,8.611614
min,0.108073,0.015217,0.000947,4.055916e-08,479.905803,2.161498e+03,499.577101,1.295350e+03,673.906438,1.130834e+03,...,-27.932220,1.531856,-20.749746,3.445751,-27.359076,3.147764,-35.614895,0.253587,0.000000,3.000000
25%,0.316037,0.079820,0.083223,6.285820e-04,1634.097151,1.228336e+05,1890.204723,4.941410e+04,3389.905912,5.562385e+05,...,-7.948162,29.821222,-2.524088,29.405123,-5.734853,30.384863,-4.012720,29.925747,6.000000,9.000000
50%,0.385163,0.085137,0.120488,1.500287e-03,2211.777107,2.643931e+05,2233.071917,9.037122e+04,4634.773513,1.155826e+06,...,-4.444725,42.235080,0.730935,41.686157,-2.700388,43.264107,-1.045194,44.173588,12.000000,15.000000
75%,0.442906,0.091154,0.175334,3.113760e-03,2713.457812,5.612200e+05,2590.295338,1.578929e+05,5597.307692,2.251497e+06,...,-0.731065,61.467625,3.871771,61.854340,0.521315,65.165130,2.192562,68.000710,21.000000,24.000000
max,0.751176,0.120717,0.440458,3.237973e-02,5432.278842,4.801847e+06,3708.279664,1.237131e+06,9486.121357,1.293661e+07,...,33.869503,523.140560,36.923035,628.774400,31.367567,1147.502400,34.130856,914.816200,27.000000,30.000000


Check for duplicates. 

In [173]:
def check_duplicates_or_nulls(data):
    # Check for duplicates
    duplicates = data.duplicated().sum() > 0
    
    # Check for any null values
    nulls = data.isnull().sum().sum() > 0
    
    # Return true if either condition met
    return duplicates or nulls

# Assuming csv_file_path is a string representing a file path, 
df = pd.read_csv('all_genres_audio_features.csv')  # Read the CSV file
if check_duplicates_or_nulls(data):
    print('Duplicates or null values found in data')
else:
    print('No duplicates or null values found in data')


Duplicates or null values found in data


## Shuffle and Split data

Split data into training and testing sets (90/10).

In [174]:


# Shuffle and split the data into training and testing sets
print("Shuffling and splitting the data into training and testing sets...")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['genre'])

Shuffling and splitting the data into training and testing sets...


## Begin Preprocessing

Apply the low pass filter through use of a moving average with a window size of 3.

In [175]:
# Function to clean and convert data
def clean_and_convert(df):
    feature_columns = df.columns.difference(['filename', 'start', 'end', 'genre'])
    for column in feature_columns:
        # Convert string representations of numbers or lists to actual numeric values
        df[column] = df[column].apply(lambda x: float(x.strip('[]')) if isinstance(x, str) and x.startswith('[') and x.endswith(']') else x)
    return df

# Apply moving average filter
def apply_moving_average_filter(df, window_size=3):
    df = clean_and_convert(df)
    
    feature_columns = df.columns.difference(['filename', 'start', 'end', 'genre'])
    df[feature_columns] = df[feature_columns].rolling(window=window_size, min_periods=1).mean()
    return df

train_df = apply_moving_average_filter(train_df)
test_df = apply_moving_average_filter(test_df)


## Z Score normalization is done to the extracted features 

Using StandardScaler to properly normalize the individual features for the train and test set, converting back to data frames upon completion. 

In [176]:

# Apply z-score normalization
def apply_zscore_normalization(train_df, test_df):
    feature_columns = train_df.columns.difference(['filename', 'start', 'end', 'genre'])
    scaler = StandardScaler()
    
    # Fit the scaler on the training data
    train_df[feature_columns] = scaler.fit_transform(train_df[feature_columns])
    
    # Transform the testing data
    test_df[feature_columns] = scaler.transform(test_df[feature_columns])
    
    return train_df, test_df

train_df, test_df = apply_zscore_normalization(train_df, test_df)

## Separate features and labels for training and testing sets

Must make sure that the labels and features themselves remain separate. 

In [177]:
# Separate features and labels for training and testing sets
X_train = train_df.drop(columns=['filename', 'start', 'end', 'genre'])
y_train = train_df['genre']
X_test = test_df.drop(columns=['filename', 'start', 'end', 'genre'])
y_test = test_df['genre']

## Save train and test data

save data to separate csv to visualize before feeding into the model

In [180]:
# Save the training and testing sets to separate CSV files
train_features_csv_path = '/Users/lchilly/Desktop/Astra/music-genre-classification/train_audio_features.csv'
train_labels_csv_path = '/Users/lchilly/Desktop/Astra/music-genre-classification/train_audio_labels.csv'
test_features_csv_path = '/Users/lchilly/Desktop/Astra/music-genre-classification/test_audio_features.csv'
test_labels_csv_path = '/Users/lchilly/Desktop/Astra/music-genre-classification/test_audio_labels.csv'

X_train.to_csv(train_features_csv_path, index=False)
y_train.to_csv(train_labels_csv_path, index=False)
X_test.to_csv(test_features_csv_path, index=False)
y_test.to_csv(test_labels_csv_path, index=False)
print("Training and testing CSV files generation completed.")

Training and testing CSV files generation completed.


## Run model on Gradient Boosting Machine Model

In [182]:
# Load datasets
train_features_df = pd.read_csv(train_features_csv_path)
train_labels_df = pd.read_csv(train_labels_csv_path)
test_features_df = pd.read_csv(test_features_csv_path)
test_labels_df = pd.read_csv(test_labels_csv_path)

# Extract features and labels
X_train = train_features_df
y_train = train_labels_df['genre']
X_test = test_features_df
y_test = test_labels_df['genre']

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the Gradient Boosting model with a manually set learning rate and other parameters
gb_model = GradientBoostingClassifier(
    n_estimators=100,       # Number of boosting stages
    learning_rate=0.05,     # Learning rate shrinks the contribution of each tree
    max_depth=4,            # Maximum depth of the individual trees
    min_samples_split=10,   # Minimum number of samples required to split an internal node
    min_samples_leaf=2,     # Minimum number of samples required to be at a leaf node
    subsample=0.9,          # Fraction of samples used for fitting the individual base learners
    random_state=42         # Ensures reproducibility
)

# Train the model
gb_model.fit(X_train_scaled, y_train_encoded)

# Predict on the test set
y_pred = gb_model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test_encoded, y_pred)
report = classification_report(y_test_encoded, y_pred, target_names=label_encoder.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)



Accuracy: 0.2954431647471207
Classification Report:
              precision    recall  f1-score   support

       blues       0.28      0.27      0.27       200
   classical       0.37      0.52      0.43       200
     country       0.23      0.19      0.21       199
       disco       0.30      0.27      0.28       200
      hiphop       0.25      0.20      0.22       200
        jazz       0.29      0.26      0.28       198
       metal       0.33      0.37      0.35       200
         pop       0.33      0.42      0.37       200
      reggae       0.29      0.33      0.31       200
        rock       0.20      0.14      0.16       200

    accuracy                           0.30      1997
   macro avg       0.29      0.30      0.29      1997
weighted avg       0.29      0.30      0.29      1997



Model is intialized, trained and test with accuracy scores displayed. 

## Run Model on Support Vector Machine

In [183]:

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the SVM model with GridSearchCV to find the best parameters
svm_model = SVC()

# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

# Use GridSearchCV to find the best parameters
grid = GridSearchCV(svm_model, param_grid, refit=True, verbose=2)
grid.fit(X_train_scaled, y_train_encoded)

# Get the best model from GridSearchCV
best_svm_model = grid.best_estimator_

# Predict on the test set
y_pred = best_svm_model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test_encoded, y_pred)
report = classification_report(y_test_encoded, y_pred, target_names=label_encoder.classes_)

print(f"Best Parameters: {grid.best_params_}")
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  11.4s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  14.7s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  10.5s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  11.4s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  10.3s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  10.5s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  10.9s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  10.8s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  10.8s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  10.9s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=  10.2s
[CV] END ......................C=0.1, gamma=0.01

## Accuracy Testing

Check the model to ensure it is doing well in terms of classifications.

## Cross-validation 

To ensure that the model is functioning well, cross-validation is done with an average result shown. 